#### Setup

In [ ]:
# Run UMAPs, classifiers, and distance metrics
    # input-data/
    # mbatch-out/
    # vae-out/

    # target outdir: ..results/<bnchmrk-phs>/r5/r5-fgs/


In [ ]:
%whos

##### dt and logreg devel

In [ ]:
from sklearn import tree
# clf = tree.DecisionTreeClassifier()

In [ ]:
from sklearn.linear_model import LogisticRegression
# clf = LogisticRegression(random_state=0).fit(X, y)

#### Imports

In [ ]:
# Data handling
import pandas as pd
import glob as glob
import itertools
import time

In [ ]:
# RFE
from sklearn.feature_selection import RFE
from sklearn.svm import SVR

In [ ]:
# RF
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import statistics
import matplotlib.pyplot as plt

# MMD
import numpy as np
from sklearn.metrics.pairwise import rbf_kernel

# Dstnc, othr
import tensorflow as tf
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import cdist

# UMAP
import umap
import os
from matplotlib.lines import Line2D

In [ ]:
%whos

#### Start

In [ ]:
systems = 'cell-line+CPTAC' # add beat aml, cptac
modality = 'transcriptomics'

#### Input, mbatch, vae file reads

##### cancers list template

In [ ]:
cancer_types = ['breast-ductal',
    'breast-lobular',
    'breast-nos',
    'colon-adeno',
    'glioblastoma',
    'head-neck',
    'lung-adeno',
    'pancreatic-adeno',
    'renal-clear-cell']

##### r_n

In [ ]:
r = 'r5'

##### One cancer file sets

In [ ]:
# Set input (raw data) to file set n
phase = 'input-data'
phase_name = 'Input data'
pths_n = sorted(glob.glob('../results/input-data/'+r+'/'+r+'-fls/*'))
print('File count: ', len(pths_n))

###### Mbatch

In [ ]:
# Set mbatch to file set n
phase = 'mbatch-out'
phase_name = 'Mbatch corrected'
pths_n = sorted(glob.glob('../results/mbatch-out/'+r+'/'+r+'-fls/*'))
print('File count: ', len(pths_n))

In [ ]:
pths_n[0]

###### VAE

In [ ]:
# Set vae to file set n
phase = 'vae-out'
phase_name = 'VAE corrected'
pths_n = sorted(glob.glob('../results/vae-out/'+r+'/'+r+'-fls/*.tsv'))
print('File count: ', len(pths_n))

In [ ]:
pths_n[0]

In [ ]:
# Base the string splits off of the last slash, next put ..tAML_25.. to tAML.25

###### Check

In [ ]:
pths_n

##### One cancer MMD

###### mmd func

In [ ]:
def compute_mmd(bio_mrk_1, bio_mrk_2, gamma=.0001):
    K_XX = rbf_kernel(bio_mrk_1, bio_mrk_1, gamma=gamma)
    K_XY = rbf_kernel(bio_mrk_1, bio_mrk_2, gamma=gamma)
    K_YY = rbf_kernel(bio_mrk_2, bio_mrk_2, gamma=gamma)

    m = bio_mrk_1.shape[0]
    n = bio_mrk_2.shape[0]

    mmd = (np.sum(K_XX) - np.trace(K_XX)) / (m * (m - 1))
    mmd += (np.sum(K_YY) - np.trace(K_YY)) / (n * (n - 1))
    mmd -= 2 * np.sum(K_XY) / (m * n)

    # Ensure the MMD value is non-negative
    mmd = np.maximum(mmd, 0)

    return np.sqrt(mmd)

###### MMD main 

In [ ]:
pwd

In [ ]:
ls ../results/devel/

In [ ]:
pths_n = [
    #'mdl-sys-bnchmrk/results/devel/colon-adeno_transcriptomics_cell-line+CPTAC_history.tsv',
'../results/devel/colon-adeno_transcriptomics_cell-line+CPTAC.latent_space.tsv']

In [ ]:
pth_n

In [ ]:
print(phase)

# Get distances 
mmd = []
# cos_sim = []
axs_lst = [] # axes means of scipy cdist
c_lst = []
m_lst = []
s_lst = []
for pi, pth_n in enumerate(pths_n):
    print(pi)
    cancer = pth_n.split('/')[-1].split('_')[0]
    modality = pth_n.split('/')[-1].split('_')[1]
    systems = pth_n.split('/')[-1].split('_')[2].split('.')[0]

    c_lst.append(cancer)
    m_lst.append(modality)
    s_lst.append(systems)
    
    df_n = pd.read_csv(
        pth_n, sep = '\t', index_col = 0)
    ab = df_n.System.unique()
    a = df_n[df_n.System == ab[0]]
    b = df_n[df_n.System == ab[1]]
    mmd.append(compute_mmd(a.iloc[:, 2:], b.iloc[:, 2:]))
    axs_lst.append(
        (np.mean(cdist(a.iloc[:, 2:], b.iloc[:, 2:],
                       metric='euclidean'), axis=1).mean(),
         np.mean(cdist(a.iloc[:, 2:], b.iloc[:, 2:],
                       metric='euclidean'), axis=0).mean()))
    # break
dstncs = pd.DataFrame({'mmd': mmd,
                             'cancer': c_lst,
                             'modality': m_lst,
                             'systems': s_lst,
                             'axs_mns': axs_lst})
# df_n

In [ ]:
dstncs

In [ ]:
# ^ DANN

In [ ]:
vae_dstncs = dstncs

In [ ]:
vae_dstncs.to_csv(
    '../results/stats-out/vae_dstncs_r5.tsv', sep = '\t')

In [ ]:
mbatch_dstncs

##### mbatch

In [ ]:
mbatch_dstncs = dstncs

In [ ]:
mbatch_dstncs.to_csv(
    '../results/stats-out/mbatch_dstncs_r5.tsv', sep = '\t')

In [ ]:
mmd # mbatch, align with cancer type / modality coverage

In [ ]:
mbatch_mmd_0001 = mmd

##### Input dstncs

In [ ]:
input-data

In [ ]:
inpt_axs_lst

In [ ]:
inpt_axs_lst = axs_lst

In [ ]:
input_mmd_0001 = mmd_0001

In [ ]:
mmd_0001 = mmd

In [ ]:
mmd_001 = mmd

In [ ]:
mmd_01 = mmd

In [ ]:
input_dstncs = pd.DataFrame({'mmd': mmd, # gamma = 1
                             'cancer': c_lst,
                             'modality': m_lst,
                             'systems': s_lst})

In [ ]:
input_dstncs['mmd'] = input_mmd_0001

In [ ]:
input_dstncs.to_csv(
    '../results/stats-out/input_dstncs_r5.tsv', sep = '\t')

In [ ]:
input_dstncs.head(1)

In [ ]:
input_dstncs['axs_mns'] = inpt_axs_lst

#### Read stats-out

In [ ]:
ls ../results/stats-out/

In [ ]:
so_pths = glob.glob('../results/stats-out/*')

In [ ]:
so_pths

In [ ]:
df_stck = pd.DataFrame()
for pth_n in so_pths:
    print(pth_n)
    phase = pth_n.split('/')[-1].split('_')[0]
    df_n = pd.read_csv(
        pth_n, sep = '\t', index_col = 0)
    df_n.insert(0, 'phase', phase)
    df_stck = pd.concat([df_stck, df_n], axis = 0)
    # break

In [ ]:
df_stck.iloc[:, :5].to_csv(
    '../results/stats-out/mmd_stack_r5.tsv', sep = '\t')

In [ ]:
pth_n = '../results/stats-out/mmd_stack_r5.tsv'
df_stack = pd.read_csv(
        pth_n, sep = '\t', index_col = 0)

In [ ]:
df_stack.phase.value_counts()

##### mbatch

In [ ]:
m_stack = df_stack[df_stack.phase == 'mbatch'].copy()

In [ ]:
m_stack['Benchmark'] = m_stack.cancer + '_' + m_stack.modality + '_' + m_stack.systems

In [ ]:
m_stack

In [ ]:
input_stack = i_stck

In [ ]:
input_stack.head(3)

In [ ]:
subset_input_stack = input_stack[
    (input_stack['modality'] == 'transcriptomics') & input_stack['cancer'].isin(m_stack['cancer'])
].copy()

In [ ]:
len(subset_input_stack)

In [ ]:
m_stack.Benchmark.value_counts()

In [ ]:
subset_input_stack.reset_index(drop = True, inplace = True)

In [ ]:
subset_input_stack.Benchmark = m_stack.Benchmark

In [ ]:
subset_input_stack

In [ ]:
inp_mbatch_stack = pd.concat([subset_input_stack, m_stack], axis = 0)

In [ ]:
# Create a new DataFrame showing the mmd for each Benchmark and phase
df_pivot_m = inp_mbatch_stack.pivot_table(index="Benchmark", columns="phase", values="mmd")

In [ ]:
phase 

In [ ]:
phase = 'mbatch'
sns.heatmap(
    df_pivot_m, annot=True, cmap="coolwarm")  # Replace "coolwarm" with your preferred colormap

# Customize the heatmap
plt.title("Maximum mean discrepency by system")
plt.xlabel("Columns")
plt.ylabel("Rows")
plt.savefig('../results/input-'+phase+'_mmd_heatmap.png')
plt.show()

##### Input and VAE

In [ ]:
i_stck = df_stck[df_stck.phase == 'input'].copy()

In [ ]:
i_stck['Benchmark'] = i_stck.cancer + '_' + i_stck.modality + '_' + i_stck.systems

In [ ]:
v_stck = df_stck[df_stck.phase == 'vae'].copy()

In [ ]:
v_stck['Benchmark'] = v_stck.cancer + '_' + v_stck.modality + '_' + v_stck.systems

In [ ]:
v_stck.Benchmark

In [ ]:
pd.concat([i_stck, v_stck], axis = 0).to_csv(
    '../results/stats-out/inp_vae_stack_r5.tsv', sep = '\t')

In [ ]:
inp_vae_stack = pd.concat([i_stck, v_stck], axis = 0)

In [ ]:
inp_vae_stack.head(3)

In [ ]:
inp_vae_stack.tail(3)

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df_pivot

In [ ]:
sns.heatmap(
    df_pivot, annot=True, cmap="coolwarm")  # Replace "coolwarm" with your preferred colormap

# Customize the heatmap
plt.title("Maximum mean discrepency by system")
plt.xlabel("Columns")
plt.ylabel("Rows")
plt.savefig('../results/input-vae_mmd_heatmap.png')
plt.show()

In [ ]:
df_pivot

In [ ]:
# Create a new DataFrame showing the mmd for each Benchmark and phase
df_pivot = inp_vae_stack.pivot_table(index="Benchmark", columns="phase", values="mmd")

# Create the bar plot
sns.barplot(data=df_pivot)
plt.xlabel("Benchmark")
plt.ylabel("MMD")
plt.title("MMD for Input and VAE Phases of Each Benchmark")
plt.show()

##### dstnc devel

In [ ]:
import numpy as np
from scipy.spatial.distance import cdist

In [ ]:
points = np.array([[1, 2, 3],
                   [4, 5, 6],
                   [2, 0, 1]])

distances = cdist(points, points, metric='euclidean')
print(distances)

In [ ]:
cdist(a.iloc[:, 2:], b.iloc[:, 2:], metric='euclidean')

In [ ]:
np.mean(cdist(a.iloc[:, 2:], b.iloc[:, 2:], metric='euclidean'), axis=1).mean()

In [ ]:
np.mean(cdist(a.iloc[:, 2:], b.iloc[:, 2:], metric='euclidean'), axis=0).mean()

In [ ]:
# mean, auc?

In [ ]:
# Pairwise samples or features
tf.keras.losses.cosine_similarity(
    y_true, y_pred, axis=-1
)

#### Pairwise file sets

###### Setup path checks

In [ ]:
ls ../results/input-data/r5/

In [ ]:
pths_n = glob.glob('../results/input-data/r5/r5-fls/*')

###### prot and trans paths

In [ ]:
len(pths_n)

In [ ]:
prot_pths_n = glob.glob('../results/input-data/r5/r5-fls/*_proteomics_*')

In [ ]:
len(prot_pths_n)

In [ ]:
tran_pths_n = glob.glob('../results/input-data/r5/r5-fls/*_transcriptomics_*')
len(tran_pths_n)

###### get_pairs function

In [ ]:
def get_pairs(pths_n):
    prot_list = []
    tran_list = []
    for file in pths_n:
        if 'proteomics' in file:
            prot_list.append(file)
        elif 'transcriptomics' in file:
            tran_list.append(file)
    tran_comb = list(itertools.combinations(tran_list, 2))
    prot_comb = list(itertools.combinations(prot_list, 2))
    return tran_comb, prot_comb

In [ ]:
ls ../results/input-data/r5/r5-fls/

###### Input pairwise cancer files

In [ ]:
# Set input (raw data) to file set n
phase = 'input-data'
phase_name = 'Input data'

pths_n = sorted(glob.glob('../results/'+phase+'/'+r+'/'+r+'-fls/*'))
print('File count: ', len(pths_n))

tran_comb, prot_comb = get_pairs(pths_n)
print('Trscrptmc, Prtmc comb lens: ', len(tran_comb), len(prot_comb))

# First and last files
print(pths_n[0].split('/')[-1]+'\n', pths_n[-1].split('/')[-1])

In [ ]:
tran_cl_cp = []
for tup in tran_comb:
    # print(tup)
    if 'cell-line+CPTAC' in tup[0] and 'cell-line+CPTAC' in tup[1]:
        # print('cl cp pair')
        tran_cl_cp.append(tup)
    # break

In [ ]:
tran_comb

In [ ]:
# Observation: 153 pairwise combos, transcriptomics; from 18 input files

In [ ]:
len(tran_cl_cp)

In [ ]:
tran_cl_cp[0]

###### Mbatch get_pairs

In [ ]:
# mBatch to file set n
phase = 'mbatch-out'
phase_name = 'Mbatch corrected'

file_set_n = sorted(glob.glob('../results/'+phase+'/'+r+'/'+r+'-fls/*'))
print('File count: ', len(file_set_n))

tran_comb, prot_comb = get_pairs(file_set_n)
print('Trscrptmc, Prtmc comb lens: ', len(tran_comb), len(prot_comb))

# First and last files
print(file_set_n[0].split('/')[-1]+'\n', file_set_n[-1].split('/')[-1])

###### VAE get_pairs

In [ ]:
# VAE to file set n
phase = 'vae-out'
phase_name = 'VAE corrected'

file_set_n = sorted(glob.glob('../results/'+phase+'/'+r+'/'+r+'-fls/*'))
print('File count: ', len(file_set_n))

tran_comb, prot_comb = get_pairs(file_set_n)
print('Trscrptmc, Prtmc comb lens: ', len(tran_comb), len(prot_comb))

# First and last files
print(file_set_n[0].split('/')[-1]+'\n', file_set_n[-1].split('/')[-1])

In [ ]:
phase

##### Infinity vals

In [ ]:
strctrd_n

In [ ]:
strctrd_n.isin([np.inf, -np.inf]).sum().sum()

In [ ]:
inf_count = f"{df_lite.isin([np.inf, -np.inf]).sum().sum():,}"

#### Classifier, one cancer

##### Clf main

###### RFE

In [ ]:
pths_n[0]

In [ ]:
pth_n

In [ ]:
mdl-sys-bnchmrk/results/devel/colon-adeno_transcriptomics_cell-line+CPTAC.latent_space.tsv

    # Evaluation file read and write naming and plot labeling and naming

r6

# One cancer file naming convention
mdl-sys-bnchmrk/results/devel/colon-adeno_transcriptomics_cell-line+CPTAC.vae-out.250-ltnt-dim_12-epchs.tsv
mdl-sys-bnchmrk/results/devel/colon-adeno_transcriptomics_cell-line+CPTAC.mbatch-out.tsv
mdl-sys-bnchmrk/results/devel/colon-adeno_transcriptomics_cell-line+CPTAC.input-data.tsv

# Two cancer file naming convention
mdl-sys-bnchmrk/results/devel/lung-adeno+colon-adeno_transcriptomics_cell-line+CPTAC.vae-out.250-ltnt-dim_12-epchs.tsv
mdl-sys-bnchmrk/results/devel/pancreatic+colon-adeno_transcriptomics_cell-line+CPTAC.mbatch-out.tsv
mdl-sys-bnchmrk/results/devel/breast-lobular+colon-adeno_transcriptomics_cell-line+CPTAC.input-data.tsv

In [ ]:

    # scatter_plot_design - restrict to transcriptomics, cell line and CPTAC
    # y-axis is the cancer type score - 21 pairs
    # x-axis is the system score - which system?
    # plot both mbatch and vae on same plot
    # connect the dots - scatter plot done

    # idea - get pairs from mBatch to drive the file read
    # predict each cancer against a single archetypal or sythetic data set
        # Standard prediction method, gen a custom synth benchmarking set for each
        # system pair within each tissue?

In [ ]:
benchmark

In [ ]:
# Single cancer setup, can only eval sys on single cancer files
    # Does is make sense to evaluate system on cancer pairwise files?

In [ ]:
pwd

In [ ]:
ls ../results/

In [ ]:
ls ../results/mbatch-out/r5/r5-fls/

In [ ]:
ls ../results/vae-out/r5/r5-fls/

In [ ]:
pth_n[0]

In [ ]:
def tts(X, y):
    X_train, X_test, y_train, y_test = train_test_split(
                                                X, y_sys)
    return X_train, X_test, y_train, y_test

In [ ]:
for pi, pth_n in enumerate(tran_cl_cp): # based on input paths
    # print(pi)
    # print(pth_n)
    # Get the benchmarks (defined by tissue type)
    b1 = pth_n[0].split('/')[-1].split('.')[0] # input r5
    b2 = pth_n[1].split('/')[-1].split('.')[0] # input r5
    
    # Get two corresponding MBatch out paths
    mb_pth1 = '../results/mbatch-out/r5/r5-fls/'+b1+'.tsv'
    mb_pth2 = '../results/mbatch-out/r5/r5-fls/'+b2+'.tsv'

    vae_pth1 = '../results/vae-out/r5/r5-fls/'+b1+'_250-ltnt-dim_12-epchs.tsv'
    vae_pth2 = '../results/vae-out/r5/r5-fls/'+b2+'_250-ltnt-dim_12-epchs.tsv'    
    # print(mb_pth1)
    # print(mb_pth2)
    
    # Read two corresponding MBatch out files
    mb_1 = pd.read_csv(
        mb_pth1, sep = '\t', index_col = 0)
    Xm1 = mb_1.iloc[:, 2:]
    yms1 = mb_1.System
    
    for xf_i in list(range(1, 5)):
        X_train, X_test, y_train, y_test = tts(Xm1, yms1)
            
    mb_2 = pd.read_csv(
        mb_pth2, sep = '\t', index_col = 0)
    Xm2 = mb_1.iloc[:, 2:]
    yms2 = mb_1.System
    
    for xf_i in list(range(1, 5)):
        X_train, X_test, y_train, y_test = tts(Xm2, yms2)
    
    vae_1 = pd.read_csv(
        vae_pth1, sep = '\t', index_col = 0)
    Xvae1 = mb_1.iloc[:, 2:]
    yvaeSys1 = mb_1.System # end two-cancer setup, end ump_clf

                            # Go to single classifier Mbatch vs VAE

    for xf_i in list(range(1, 5)):
        X_train, X_test, y_train, y_test = tts(Xm2, yms2)    
    
    vae_2 = pd.read_csv(
        vae_pth2, sep = '\t', index_col = 0)

    # Make three Mbatch system predictions
        # Start with one

    ab_1 = mb_1.System.unique()
    ab_2 = mb_2.System.unique()

    mb_1a = mb_1[mb_1.System == ab_1[0]]
    mb_1b = mb_1[mb_1.System == ab_1[1]]

    mb_2a = mb_2[mb_2.System == ab_2[0]]
    mb_2b = mb_2[mb_2.System == ab_2[1]]

    ab_1 = vae_1.System.unique()
    ab_2 = vae_2.System.unique()

    vae_1a = vae_1[vae_1.System == ab_1[0]]
    vae_1b = vae_1[vae_1.System == ab_1[1]]

    vae_2a = vae_2[vae_2.System == ab_2[0]]
    vae_2b = vae_2[vae_2.System == ab_2[1]]

    for xf_i in list(range(1, 5)):
        X_train, X_test, y_train, y_test = train_test_split(
                                                X, y_sys)
    
    # Make Mbatch tissue prediction

    # Repeat for VAE - non dot in file name

    # To x and y on scatter

    print('end')
    break

In [ ]:
mb_1

In [ ]:
df_n

In [ ]:
# RFE

In [ ]:
rfe = RFE(estimator=estimator, n_features_to_select=10, step=10)
file_n = pd.read_csv(pth, sep = '\t', index_col = 0) # read validation split file
X = file_n.iloc[:, 1:]
y = file_n.iloc[:, 0]
rfe.fit(X, y)
mask = rfe.support_
vs_dict[vs] = json.dumps(list(X.columns[mask]))

##### Devel, scatter crossfolds

In [ ]:
for pi, pth_n in enumerate(tran_cl_cp): # based on input paths
    # print(pi)
    # print(pth_n)
    # Get the benchmarks (defined by tissue type)
    b1 = pth_n[0].split('/')[-1].split('.')[0] # input r5
    b2 = pth_n[1].split('/')[-1].split('.')[0] # input r5
    
    # Get two corresponding MBatch out paths
    mb_pth1 = '../results/mbatch-out/r5/r5-fls/'+b1+'.tsv'
    mb_pth2 = '../results/mbatch-out/r5/r5-fls/'+b2+'.tsv'

    vae_pth1 = '../results/vae-out/r5/r5-fls/'+b1+'_250-ltnt-dim_12-epchs.tsv'
    vae_pth2 = '../results/vae-out/r5/r5-fls/'+b2+'_250-ltnt-dim_12-epchs.tsv'    
    # print(mb_pth1)
    # print(mb_pth2)
    
    # Read two corresponding MBatch out files
    mb_1 = pd.read_csv(
        mb_pth1, sep = '\t', index_col = 0)
    mb_2 = pd.read_csv(
        mb_pth2, sep = '\t', index_col = 0)

    vae_1 = pd.read_csv(
        vae_pth1, sep = '\t', index_col = 0)
    vae_2 = pd.read_csv(
        vae_pth2, sep = '\t', index_col = 0)

    # Make three Mbatch system predictions
        # Start with one

    ab_1 = mb_1.System.unique()
    ab_2 = mb_2.System.unique()

    mb_1a = mb_1[mb_1.System == ab_1[0]]
    mb_1b = mb_1[mb_1.System == ab_1[1]]

    mb_2a = mb_2[mb_2.System == ab_2[0]]
    mb_2b = mb_2[mb_2.System == ab_2[1]]

    ab_1 = vae_1.System.unique()
    ab_2 = vae_2.System.unique()

    vae_1a = vae_1[vae_1.System == ab_1[0]]
    vae_1b = vae_1[vae_1.System == ab_1[1]]

    vae_2a = vae_2[vae_2.System == ab_2[0]]
    vae_2b = vae_2[vae_2.System == ab_2[1]]

    for xf_i in list(range(1, 5)):
        X_train, X_test, y_train, y_test = train_test_split(
                                                X, y_sys)
    
    # Make Mbatch tissue prediction

    # Repeat for VAE - non dot in file name

    # To x and y on scatter

    print('end')
    break

##### Classifier, cancer type pairs

In [ ]:
phase_name

In [ ]:
# Cancer combos n - to scatter within each modality 
    # run on three phases: ../results/input-data,
                        #  ../results/mbatch-out, 
                        #  ../results/vae-out
start = time.time()

# phase
print(phase_name)

# Transcriptomics toggle
cmb_lst_n = tran_comb

# Proteomics togglea
# cmb_lst_n = prot_comb

# Joint embedding runs
# cmb_lst_n = je_comb
'''
end toggles
'''
for cmb_n, pth_cmb_n in enumerate(cmb_lst_n):
    # print(cmb_n, fl_pths)
    
    df_n1 = pd.read_csv(
        pth_cmb_n[0], sep = '\t', index_col = 0)
    cncr_1 = pth_cmb_n[0].split('_')[-3].split('/')[-1]
    print(cncr_1)
    
    inf_count = f"{df_n1.isin([np.inf, -np.inf]).sum().sum():,}"
    print(inf_count)

    df_n1.isin([np.inf, -np.inf]).sum().sum() #
    
    print(inf_count)
    df_n2 = pd.read_csv(
        pth_cmb_n[1], sep = '\t', index_col = 0)
        
    cncr_2 = pth_cmb_n[1].split('_')[-3].split('/')[-1]
    inf_count = f"{df_n2.isin([np.inf, -np.inf]).sum().sum():,}"
    print(inf_count)
    
    # print('check')
    # break

    x = clf_main(phase, phase_name, pth_cmb_n, df_n1, df_n2)
    # mmd_main(fl_n, file_set_n)
    # umap_main(fl_n, file_set_n)
    # return
    break
time.time() - start

In [ ]:
x

In [ ]:
def clf_main(phase, phase_name, pth_cmb_n, df_n1, df_n2):

    # print(strctrd.value_counts(strctrd.System))
                # to plot labels
    print('Start clf_main')
    # return pth_cmb_n
    modality = pth_cmb_n[0].split('_')[-2]
    # return modality
    cncr_1 = pth_cmb_n[0].split('_')[-3].split('/')[-1]
    cncr_2 = pth_cmb_n[1].split('_')[-3].split('/')[-1]
    # return phase, modality, cancer1, cancer2
    if df_n1.isna().sum().sum() > 0:
        print('strctr1 NaN values: ', cncr_1, phase)
        return # To next file
    if df_n2.isin([np.inf, -np.inf]).sum().sum() > 0:
        print('strct2 inf values: ', cncr_1, phase)
        return # To next file    

    if df_n2.isna().sum().sum() > 0:
        print('strct2 NaN values: ', cncr_2, phase)
        return # To next file
    if df_n2.isin([np.inf, -np.inf]).sum().sum() > 0:
        print('strct2 inf values: ', cncr_2, phase)
        return # To next file

    two_cncer = pd.concat([df_n1, df_n2], axis = 0,
                       join = 'inner')
    X =two_cncer.iloc[:, 2:]
    y_sys = two_cncer.System
    y_cncr = two_cncer.Cancer_type # to Cross val index matching
    # return X
    sys_scores = [] # between these two cancers
    cncr_scores = [] # between these two cancers
    # return fl_pths, cancer1, cancer2
    
    clf_name = 'rnd-frst'
    # clf_name = 'dt'
    # clf_name = 'lg_rg'

    return clf_name
    for xf_i in list(range(1,10)):
    # for xf_i in list(range(1,30)):
        X_train, X_test, y_train, y_test = train_test_split(
                                                X, y_sys)
        # System                                      ^
        clf = RandomForestClassifier(
                    max_depth=2,
                    random_state=0).fit(
                                    X_train, y_train)
        
        # clf = tree.DecisionTreeClassifier().fit(X_train, y_train)
        # clf = LogisticRegression(random_state=0).fit(X_train, y_train)
        
        score = f1_score(y_test, clf.predict(X_test), average = 'weighted')
        sys_scores.append(score)

        # Cross val index matching           y_c_typ:
        y_train = y_cncr[y_cncr.index.isin(y_train.index)]
        y_test = y_cncr[y_cncr.index.isin(y_test.index)]

        # Cancer type
        clf = RandomForestClassifier(
                    max_depth=2,
                    random_state=0).fit(
                                    X_train, y_train)
        
        score = f1_score(y_test, clf.predict(X_test), average = 'weighted')
        cncr_scores.append(score)
    
    #     break
    # return cncr_scores
    # for score_list in 
    sys_mean = statistics.mean(sys_scores)
    sys_err = statistics.stdev(sys_scores)

##### Bar plot templates

In [ ]:
    # fig, ax = plt.subplots()
    # plt.ylim(0, 1)
    
    # # System barplot
    # bar_s = ax.bar('sys', sys_mean, yerr=sys_err, capsize = 7, label='System')
    # plt.title(phase_name+' '+cancer1+'+'+cancer2+
    #     '_'+modality)
    
    # # Legend
    # ax.legend()
    # plt.savefig(
    #     '../results/'+phase+'/'+r+'/'+r+'_fgs/'+phase+'_'+cancer1+'+'+cancer2+
    #     '_'+modality+'_'+systems+'_'+clf_name+'_sys.png')
    # print(cancer1+'+'+cancer2, modality, ' to disk')
    # print(' ')
    # plt.close()
    # cncr_mean = statistics.mean(cncr_scores)
    # cncr_err = statistics.stdev(cncr_scores)
    
    # fig, ax = plt.subplots()
    # plt.ylim(0, 1)
    
    # # Cancer barplot
    # bar_c = ax.bar('sys', cncr_mean, yerr=cncr_err, capsize = 7, label='Cancer')
    # plt.title(phase_name+' '+cancer1+'+'+cancer2+
    #     '_'+modality)
    
    # # Cancer Legend
    # ax.legend()
    # plt.savefig(
    #     '../results/'+phase+'/'+r+'/'+r+'_fgs/'+phase+'_'+cancer1+'+'+cancer2+
    #     '_'+modality+'_'+systems+'_'+clf_name+'_cncr.png')
    # print(cancer1+'+'+cancer2, modality, ' to disk')
    # print(' ')
    # plt.close()    
    # # return x
    # # break

#### MMD template

In [ ]:
# order file names by cancer, modality, systems - 
    # for sorting in dirs - mult cncrs per moda, mult modas per system

In [ ]:
def compute_mmd(bio_mrk_1, bio_mrk_2, gamma=1.0):
    K_XX = rbf_kernel(bio_mrk_1, bio_mrk_1, gamma=gamma)
    K_XY = rbf_kernel(bio_mrk_1, bio_mrk_2, gamma=gamma)
    K_YY = rbf_kernel(bio_mrk_2, bio_mrk_2, gamma=gamma)

    m = bio_mrk_1.shape[0]
    n = bio_mrk_2.shape[0]

    mmd = (np.sum(K_XX) - np.trace(K_XX)) / (m * (m - 1))
    mmd += (np.sum(K_YY) - np.trace(K_YY)) / (n * (n - 1))
    mmd -= 2 * np.sum(K_XY) / (m * n)

    # Ensure the MMD value is non-negative
    mmd = np.maximum(mmd, 0)

    return np.sqrt(mmd)

##### mmd inspection

In [ ]:
for i, file in enumerate(file_set_n):
    # print(file)
    strctrd_n = pd.read_csv(
        file,
        sep = '\t', index_col = 0)
    
    # print(strctrd.value_counts(strctrd.System))
    print(' ')
    cancer = cancer_types[i]
    print(cancer)
    # break
    bio_mrk_1 = strctrd_n[strctrd_n.System=='cell_line'].iloc[:, 2:]
    bio_mrk_2 = strctrd_n[strctrd_n.System=='CPTAC'].iloc[:, 2:]
    
    mmd_score = compute_mmd(bio_mrk_1, bio_mrk_2)
    print(mmd_score)
    # break

#### UMAP

In [ ]:
# Set input (raw data) to file set n
phase = 'input_data'
phase_name = 'Input data'
file_set_n = sorted(glob.glob('../strctrd/one_cncr/*'))

In [ ]:
# UMAP on structured data, input data
print(phase_name)
for i, file in enumerate(file_set_n):
    strctrd_n = pd.read_csv(
        file,
        sep = '\t', index_col = 0)
    cancer = cancer_types[i]
    print(cancer)
    print(' ')
    # break
    features = strctrd_n.drop(['System', 'Cancer_type'], axis=1)
    system_labels = strctrd_n['System'] # data phase
    
    umap_result = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, metric='euclidean').fit_transform(features)
    unique_labels = system_labels.unique()
    
    colors = ['red', 'green']
    color_map = dict(zip(unique_labels, colors))
    
    plt.figure(figsize=(10, 8))
    
    for i in range(len(umap_result)):
        plt.scatter(umap_result[i, 0], umap_result[i, 1],
                    c=[color_map[system_labels.iloc[i]]],
                    edgecolor='k',
                    s=100,
                    label=system_labels.iloc[i] + ' - ' + system_labels.iloc[i] if i in [list(system_labels).index(x) for x in unique_labels] else "")
    
    plt.legend()
    
    plt.title(cancer+', '+phase+', '+modality, fontsize=14)
    
    plt.tight_layout()
    plt.savefig(
        '../results/'+phase+'/one_cncr/figures/'+phase+'_'+cancer+
        '_'+modality+'_'+systems+'rf.png')
    
    plt.show()
    # break

In [ ]:
# Set input (mbatch data) to file set n
phase = 'mbatch_out'
phase_name = 'Mbatch corrected'
file_set_n = sorted(glob.glob('../results/mbatch_out/one_cncr/files/*'))

In [ ]:
# UMAP on structured data, mbatch out
print(phase_name)
for i, file in enumerate(file_set_n):
    strctrd_n = pd.read_csv(
        file,
        sep = '\t', index_col = 0)
    cancer = cancer_types[i]
    print(cancer)
    print(' ')
    # break
    features = strctrd_n.drop(['System', 'Cancer_type'], axis=1)
    system_labels = strctrd_n['System'] # data phase
    
    umap_result = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, metric='euclidean').fit_transform(features)
    unique_labels = system_labels.unique()
    
    colors = ['red', 'green']
    color_map = dict(zip(unique_labels, colors))
    
    plt.figure(figsize=(10, 8))
    
    for i in range(len(umap_result)):
        plt.scatter(umap_result[i, 0], umap_result[i, 1],
                    c=[color_map[system_labels.iloc[i]]],
                    edgecolor='k',
                    s=100,
                    label=system_labels.iloc[i] + ' - ' + system_labels.iloc[i] if i in [list(system_labels).index(x) for x in unique_labels] else "")
    
    plt.legend()
    
    plt.title(cancer+', '+phase+', '+modality, fontsize=14)
    
    plt.tight_layout()

    plt.savefig(
        '../results/'+phase+'/one_cncr/figures/'+phase+'_'+cancer+
        '_'+modality+'_'+systems+'rf.png')
    plt.show()
    # break

### UMAP function

In [ ]:
# UMAP
def umap(df, modality, )
    features = df.drop(['Data_phase', 'Labels'], axis=1)
    data_phase = df['Data_phase']
    labels = df['Labels']
    
    umap_result = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, metric='euclidean').fit_transform(features)
    unique_labels = labels.unique()
    
    colors = ['red', 'green']
    color_map = dict(zip(unique_labels, colors))
    
    shape_dict = {'Original': 'o', 'Synthetic': '^'}
    
    # Begin UMAPs on n = 25
    plt.figure(figsize=(10, 8))
    
    for i in range(len(umap_result)):
        plt.scatter(umap_result[i, 0], umap_result[i, 1],
                    c=[color_map[labels.iloc[i]]],
                    marker=shape_dict[data_phase.iloc[i]],
                    edgecolor='k',
                    s=100,
                    label=labels.iloc[i] + ' - ' + data_phase.iloc[i] if i in [list(labels).index(x) for x in unique_labels] else "")
    
    labels = list(unique_labels)
    color_legend = [Line2D([0], [0], marker='s', color='w', label=label,
                           markersize=10, markerfacecolor=color) for label, color in color_map.items() if label in labels]
    
    
    first_legend = plt.legend(handles=color_legend, title='Labels', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.gca().add_artist(first_legend)
    shape_legend = [Line2D([0], [0], marker=shape, color='w', label=status,
                           markersize=10, markerfacecolor='gray') for status, shape in shape_dict.items()]
    plt.legend(handles=shape_legend, title='Data Phase', bbox_to_anchor=(1.05, 0.5), loc='upper left')
    
    plt.title(cancer+', '+data_type, fontsize=14)
    
    plt.tight_layout()
    plt.savefig(f'i_o/UMAP_{v}_{systems}_{cancer}_{modality}.png',
                bbox_inches='tight', dpi = 300) # file name, need to replace cancer name and version with auto vars
    
    plt.show()